In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import matthews_corrcoef


def create_target(value):

    if(value in ["A", "B","C","D"]):
        return "hyperthyroid condition"

    elif(value in ["E", "F", "G", "H"]):
        return "hypothyroid condition"

    elif(value in ["I", "J"]):
        return "binding protein"

    elif(value in ["L", "M", "N"]):
        return "replacement therapy"

    elif(value == "R"):
        return "discordant results"

    elif(value == "K"):
        return "general health"

    elif(value == "-"):
        return "hypothyroid condition"

    return "other"

def replaceCol(col):
    arr = []
    mapa = dict()
    n = -1
    for line in col:
        if(line == "?"):
            arr.append(-1)
        elif(line not in mapa.values):
            n+=1
            mapa.update(line, n)
            arr.append(n)
        else:
            e = mapa.get(line)
            arr.append(e)
    return arr

def naive_model_testing(train, test):

    #test 2 models, DTs and LR, and print out the results
    dtr= DecisionTreeRegressor(max_depth=5)
    dtr.fit(train.drop('target', axis=1), train['target'])

    lmr=LinearRegression()
    lmr.fit(train.drop('target', axis=1), train['target'])

   # rf_preds=rfr.predict(X_test)
    dt_preds=dtr.predict(test.drop('target', axis=1))
    lr_preds=lmr.predict(test.drop('target', axis=1))

   # print("RVE RFs: %7.4f" % explained_variance_score(y_test, rf_preds))
    print("R2 Decision Tree Regression: %7.4f" % f1_score(test['target'], dt_preds))
    print("R2 Linear Regression: %7.4f" % f1_score(test['target'], lr_preds))


data = pd.read_csv("proj-data.csv")

data['target'] = data['diagnoses'].apply(create_target)

data = data.drop(['diagnoses', '[record identification]'], axis=1)

data = data.replace('?', -1)
data = data.replace('f', 0)
data = data.replace('t', 1)
data = data.replace('M', 1)
data = data.replace('F', 0)

data['referral source:'] = data['referral source:'].astype('category').cat.codes#replaceCol(data['referral source:']) #
data['target'] = data['target'].astype('category').cat.codes#replaceCol(data['target']) #
#--------------------------------------------------- -------------------------------------



# para avaliar o modelo, vamos dividir o dataset em treino e teste


C:\Users\bakug\AppData\Local\Temp\ipykernel_20120\2319189651.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('t', 1)
C:\Users\bakug\AppData\Local\Temp\ipykernel_20120\2319189651.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('F', 0)


In [3]:

spear = data.corr(method='spearman')

d = spear['target'].abs().nlargest(30)
# print("spearman")
# print(d)

In [4]:
pearson = data.corr(method='pearson')

d = pearson['target'].abs().nlargest(30)
print("pearson")
print(d)

pearson
target                        1.000000
on thyroxine:                 0.252414
pregnant:                     0.223739
referral source:              0.184009
TT4:                          0.137815
T4U:                          0.135885
T3:                           0.103967
TT4 measured:                 0.088449
TBG:                          0.087907
T4U measured:                 0.083256
FTI measured:                 0.082986
TBG measured:                 0.081623
FTI:                          0.077356
T3 measured:                  0.074284
TSH measured:                 0.059817
TSH:                          0.049561
thyroid surgery:              0.046009
query hypothyroid:            0.045878
I131 treatment:               0.037085
query hyperthyroid:           0.031676
query on thyroxine:           0.028076
on antithyroid medication:    0.017606
sex:                          0.011166
sick:                         0.008605
lithium:                      0.005053
psych:           

In [38]:

data.filter(pearson['target'].abs().nlargest(10))

X = data.drop(['target'], axis=1)
y = data['target']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
#train, test = train_test_split(data, test_size=0.2, random_state=0)

#f1_score(y_train, y_test)

# Assuming you've imported necessary libraries and defined 'data' and 'pearson' appropriately

# Filter data based on the top correlated features
# top_correlated_features = pearson['target'].abs().nlargest(10).index
# filtered_data = data[top_correlated_features]

# # Define features (X) and target (y)
# X = filtered_data.drop(['target'], axis=1)
# y = filtered_data['target']

# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)



In [ ]:
# N, M = train.shape
# M = M-1
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(x_train,y_train)
# sfs = SequentialFeatureSelector(tree, n_features_to_select=5).set_output(transform="pandas")
# sfs.fit(X, y)

pred = tree.predict(x_test)

#get the relevant columns
# features=sfs.get_support()
# Features_selected =np.arange(M)[features]
# print("The features selected are columns: ", Features_selected)
# print(data.columns[Features_selected])

f1_score(y_test, pred, average="weighted")

In [25]:




for i in range(2, data.columns.size):

    dt = data.filter(pearson['target'].abs().nlargest(i).index)
    dt['target'] = data['target']

    X = dt.drop(['target'], axis=1)
    y = dt['target']
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    tree = DecisionTreeClassifier()
    tree.fit(x_train,y_train)
    test_pred = tree.predict(x_test)
    train_pred = tree.predict(x_train)
    print(i,": ")
    print("f1: ", f1_score(y_test, test_pred, average="weighted"))
    print("accuracy train: ", accuracy_score(y_train, train_pred))
    print("accuracy tesst: ", accuracy_score(y_test, test_pred))
    print("matthews: ",matthews_corrcoef(y_test, test_pred), "\n")
    print("colunas: ", dt.columns)



2 : 
f1:  0.7050894551649142
accuracy train:  0.8068143100511074
accuracy tesst:  0.7956403269754768
matthews:  0.0 

colunas:  Index(['target', 'on thyroxine:'], dtype='object')
3 : 
f1:  0.712620064972889
accuracy train:  0.8114139693356047
accuracy tesst:  0.7970027247956403
matthews:  0.09900077088707553 

colunas:  Index(['target', 'on thyroxine:', 'pregnant:'], dtype='object')
4 : 
f1:  0.7113129099539284
accuracy train:  0.8115843270868824
accuracy tesst:  0.7963215258855586
matthews:  0.08698611505752021 

colunas:  Index(['target', 'on thyroxine:', 'pregnant:', 'referral source:'], dtype='object')
5 : 
f1:  0.7561018447907218
accuracy train:  0.8678023850085179
accuracy tesst:  0.7915531335149864
matthews:  0.28936854917467253 

colunas:  Index(['target', 'on thyroxine:', 'pregnant:', 'referral source:', 'TT4:'], dtype='object')
6 : 
f1:  0.7840217969920056
accuracy train:  0.9747870528109029
accuracy tesst:  0.7908719346049047
matthews:  0.3919235795159162 

colunas:  Index([

In [7]:
from sklearn.calibration import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from xgboost import XGBClassifier


def present_class_statistics(y_test, y_train, train_pred, test_pred, modelo):
    print(modelo)
    print("f1: ", f1_score(y_test, test_pred, average="macro"))
    print("accuracy train: ", accuracy_score(y_train, train_pred))
    print("accuracy tesst: ", accuracy_score(y_test, test_pred))
    print("matthews: ",matthews_corrcoef(y_test, test_pred), "\n")


def modelos(X):

    scaler = MinMaxScaler()
    y = data['target']
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    print("tree")
    tree = DecisionTreeClassifier()
    tree.fit(x_train, y_train)
    test_pred = tree.predict(x_test)
    train_pred = tree.predict(x_train)
    print("accuracy train: ", accuracy_score(y_train, train_pred))
    print(classification_report(y_test, test_pred, zero_division=0.0))
    #present_class_statistics(y_test, y_train, train_pred, test_pred, "Tree")

    print("RandomForest")
    rd = RandomForestClassifier(n_estimators=150, random_state=0)
    rd.fit(x_train, y_train)
    test_pred = rd.predict(x_test)
    train_pred = rd.predict(x_train)
    print("accuracy train: ", accuracy_score(y_train, train_pred))
    print(classification_report(y_test, test_pred, zero_division=0.0))
    #present_class_statistics(y_test, y_train, train_pred, test_pred, "Random Forest")

    print("XGB")
    xgb = XGBClassifier()
    xgb.fit(x_train, y_train)
    test_pred = xgb.predict(x_test)
    train_pred = xgb.predict(x_train)
    print("accuracy train: ", accuracy_score(y_train, train_pred))
    print(classification_report(y_test, test_pred, zero_division=0.0))
    #present_class_statistics(y_test, y_train, train_pred, test_pred, "XGB")


@ignore_warnings(category=ConvergenceWarning)
def tudo():
    train, test = train_test_split(data, test_size=0.2, random_state=0)
    N, M = train.shape
    M = M-1


    models1 = {
            "Decision Tree": DecisionTreeClassifier(),
            "Random Forest": RandomForestClassifier(n_estimators=150, random_state=0),
            "XGB": XGBClassifier()
            #"KNN" : KNeighborsClassifier()
        }

    for i in range(2, data.columns.size-1):

        print(i,": \n")
        print("sfs: ")

        for j in models1:
            print(j)
            sfs = SequentialFeatureSelector(models1.get(j), n_features_to_select=i, direction="forward").set_output(transform="pandas")
            sfs.fit(train.drop('target', axis=1), train['target'])

            #get the relevant columns
            features=sfs.get_support()
            Features_selected = np.arange(M)[features]
            print("The features selected are columns: ", data.columns[Features_selected])

            X = data.filter(data.columns[Features_selected])
            modelos(X)

        dt = data.filter(pearson['target'].abs().nlargest(i).index)
        X = dt.drop(['target'], axis=1)
        print("pearson: ")
        print("colunas: ", dt.columns)
        modelos(X)

        dt2 = data.filter(spear['target'].abs().nlargest(i).index)
        X = dt2.drop(['target'], axis=1)
        print("spear: ")
        print("colunas: ", dt2.columns)
        modelos(X)

tudo()

2 : 

sfs: 
Decision Tree
The features selected are columns:  Index(['T3:', 'FTI:'], dtype='object')
tree
accuracy train:  0.941396933560477
              precision    recall  f1-score   support

           0       0.36      0.30      0.33        56
           1       0.34      0.33      0.34        30
           2       0.76      0.77      0.76        81
           3       0.49      0.57      0.52        30
           4       0.91      0.94      0.93      1168
           5       0.19      0.12      0.14        43
           6       0.34      0.25      0.29        60

    accuracy                           0.83      1468
   macro avg       0.48      0.47      0.47      1468
weighted avg       0.82      0.83      0.82      1468

RandomForest
accuracy train:  0.941396933560477
              precision    recall  f1-score   support

           0       0.50      0.36      0.42        56
           1       0.31      0.33      0.32        30
           2       0.76      0.72      0.74        

KeyboardInterrupt: 